In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import VisualBertForPreTraining, BertTokenizer
import pickle
import pandas as pd


In [126]:
# Define the training dataset
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        text = self.data['text_input'][index]
        label = self.data['label'][index]
        embedded = self.data['embedded'][index]
        
        return text, embedded, label

    def __len__(self):
        return len(self.data)



In [4]:
# Define the pre-trained Visual-Bert model
model = VisualBertForPreTraining.from_pretrained('uclanlp/visualbert-vqa-coco-pre', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [5]:

# Define the pre-trained Visual-Bert model
model = VisualBertForPreTraining.from_pretrained('uclanlp/visualbert-vqa-coco-pre', num_labels=2)
#tokenizer = VisualBertTokenizer.from_pretrained('uclanlp/visualbert-vqa-coco-pre')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#possiblement a changer le VQA 

In [129]:
# Define the training loop

def train(model, tokenizer, train_dataset, optimizer, criterion, device, batch_size, epochs):
    model.to(device)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    for epoch in range(epochs):
        running_loss = 0.0
        for text,embedded,label in train_loader:
            optimizer.zero_grad()
            text_encoded = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
            image = embedded.to(device)
            text_encoded = {k: v.to(device) for k, v in text_encoded.items()}
            label = label.to(device)
            outputs = model(inputs_embeds=text_encoded['input_ids'], visual_embeds=image, labels=label)
            loss = criterion(outputs.logits, label)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            #print(1)
            
        print('Epoch [%d] - loss: %.4f' % (epoch+1, running_loss/len(train_loader)))

In [29]:

# Define the optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()


In [19]:
df=pd.read_json("data/train.jsonl",lines=True)

l = []

with open(r"C:\Users\arman\OneDrive\Bureau\data\file.pkl","rb") as f:
    l = pickle.load(f)
f.close()
df.loc[:, 'embedded'] = l

In [121]:
essai=df.sample(20)

In [130]:
train_dataset = MyDataset(essai)
train(model, tokenizer, train_dataset, optimizer, criterion, device, batch_size=1, epochs=10)

KeyError: 'text_input'

In [ ]:
# Define the training data
train_data = [
    {'text': 'A cute cat is sleeping', 'image': '/path/to/image1.jpg', 'label': 1},
    {'text': 'A dog is playing in the park', 'image': '/path/to/image2.jpg', 'label': 0},
    {'text': 'A group of people are enjoying a meal', 'image': '/path/to/image3.jpg', 'label': 0},
    # Add more data as needed
]